In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import healpy as hp
from scipy.spatial import ConvexHull
import meshio
import scipy
from pygsp.graphs import Graph
import healpy as hp
from pyevtk.hl import unstructuredGridToVTK
from pyevtk.vtk import VtkTriangle, VtkQuad
import pyshtools

class MyGraph(Graph):
    
    def __init__(self, W, coords):
        super(MyGraph, self).__init__(W=W, coords=coords)
        

    
    def _init_vtk(self):
        from pyevtk.vtk import VtkTriangle, VtkQuad
        tri = ConvexHull(self.coords)
        self.simplices = tri.simplices
        self.conn = self.simplices.flatten()
        self.offset = np.arange(3,len(self.conn)+1, 3)
        self.ctype = np.ones(len(self.offset))*VtkTriangle.tid


    def save_signal(self, title='unstructured', pointData=None):
        from pyevtk.hl import unstructuredGridToVTK
        if pointData is not None:
            unstructuredGridToVTK(
                title,
                self.x,
                self.y,
                self.z,
                connectivity=self.conn,
                offsets=self.offset,
                cell_types=self.ctype,
                pointData={'Temp': pointData})
        else:
            unstructuredGridToVTK(
                title,
                self.x,
                self.y,
                self.z,
                connectivity=self.conn,
                offsets=self.offset,
                cell_types=self.ctype,
                pointData={'Temp': self.x})
        
    def save_mesh(self, file="mesh.xml"):
        self._init_vtk()
        cells = {"triangle": self.simplices}
        meshio.write_points_cells(
            file,
            self.coords,
            cells)
        
class FullEquiangular(MyGraph):

    sigmas = {1:1,
              2:0.3,
              4: 0.175, 
              8: 0.0525,
              16: 0.015,
              32: 0.004}
    
    def make_coords(self, bw):
        alpha = np.arange(2 * bw) * np.pi / bw
        beta = np.arange(2 * bw) * np.pi / (2. * bw)
        theta, phi = np.meshgrid(*(beta, alpha),indexing='ij')
        ct = np.cos(theta).flatten()
        st = np.sin(theta).flatten()
        cp = np.cos(phi).flatten()
        sp = np.sin(phi).flatten()
        x = st * cp
        y = st * sp
        z = ct
        coords = np.vstack([x, y, z]).T
        coords = np.asarray(coords, dtype=np.float32)
        coords = coords[2*bw-1:]
        self.x = coords[:,0]
        self.y = coords[:,1]
        self.z = coords[:,2]
        return coords

    
    def __init__(self, bw=8):
        self.bw = bw
        self.npix = 2*bw*(2*bw-1)+1
        coords = self.make_coords(bw)
        distances_squared = scipy.spatial.distance.cdist(coords, coords)**2
        try:
            sigma = self.sigmas[bw]
        except KeyError:
            sigma = 0.1
        W = np.exp(-distances_squared/sigma)
        W = W-np.diag(np.diag(W))
        super(FullEquiangular, self).__init__(W=W, coords=coords)
        self._init_vtk()

In [2]:
def to_signal(array):
    bw = array.shape[0]//2
    return array.flatten()[2*bw-1:]


def to_array(f):
    N = f.size
    bw = int((2+np.sqrt(4+16*N))//8)
    
    """From a 1-d vector to a 2D grid necessary to initiate a pyshtools.SHGrid object"""
    height, width = 2*bw, 2*bw
    array = np.zeros((height, width))  # shape=(longitude, latitude)
    f = np.append([f[0]]*(2*bw-1), f)  # correct! the first line is the North pole repeated 2bw times
    # now we need to undo the meshgrid
    assert f.size == array.size
    for n, fx in enumerate(f):
        j = n%width
        i = n//width
        array[i, j] = fx
    return array


def _filter(grid, matrix):
    array = grid.data
    f = to_signal(array)
    ### here goes the filtering ###
    f = matrix@f
    ### ----------------------- ###
    array = to_array(f, )
    return pyshtools.SHGrid.from_array(array)


def _equivariance_error(clm, max_iter, matrix, plot=False):
    """Calculates the equivariance error on one signal  only
    as average of max_iter random rotations"""
    
    grid = clm.expand()
    if plot:
        grid.plot()
    angles = np.random.uniform(low=0, high=360, size=(max_iter, 3))
    
    initial_norm = np.linalg.norm(grid.data)
    norm = np.zeros(max_iter)
    for i, [alpha, beta, gamma] in enumerate(angles):
        clm_rotated = clm.rotate(alpha, beta, gamma, degrees=True)
        grid_rotated = clm_rotated.expand()

        F_grid = _filter(grid, matrix)
        F_clm = F_grid.expand()
        RF_clm = F_clm.rotate(alpha, beta, gamma, degrees=True)
        RF_grid = RF_clm.expand()

        FR_grid = _filter(grid_rotated, matrix)
        if plot:
            FR_grid.plot()
            RF_grid.plot()
        norm[i] = np.linalg.norm(RF_grid.data-FR_grid.data)
    return np.mean(norm)/initial_norm


def equivariance_error(max_iter, matrix, plot=False):
    """Calculates the equivariance error on max_iter signals
    with the same power spectrum as average of N random rotations each"""
    N = 5
    degrees = np.arange(lmax, dtype=float)
    degrees[0] = np.inf
    power = degrees**(-2)
    norm = np.zeros(max_iter)
    for i in range(max_iter):
        clm = pyshtools.SHCoeffs.from_random(power)
        norm[i] = _equivariance_error(clm, N, matrix, plot)
    return np.mean(norm)

In [3]:
bw = 8
lmax = bw

sphere = FullEquiangular(bw=bw)
HKGL = sphere.L
HKGL_diffusion = scipy.linalg.expm(-3*HKGL)

/Users/Mart/anaconda3/envs/pdm/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:296: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/Users/Mart/anaconda3/envs/pdm/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:203: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  'is in the CSC matrix format', SparseEfficiencyWarning)


In [4]:
print(equivariance_error(100, HKGL_diffusion, plot=False))    

0.168464763993304


In [12]:
npix = 2*bw*(2*bw-1)+1

L = scipy.sparse.load_npz('../03.FEM_laplacian/equiangular/normal/matrices/stiffness_matrix_{}.npz'.format(bw))
B = scipy.sparse.load_npz('../03.FEM_laplacian/equiangular/normal/matrices/mass_matrix_{}.npz'.format(bw))
reordering_mask = np.load("../03.FEM_laplacian/equiangular/normal/15_reordering_masks/reordering_mask_{}.npy".format(bw))

L = L[reordering_mask]
B = B[reordering_mask]
L = L[:, reordering_mask]
B = B[:, reordering_mask]
B_inv = scipy.sparse.linalg.inv(B)


B_lumped_12 = np.diag(1./np.sqrt(np.sum((B.toarray()), axis=1)))
B_lumped_inv = np.diag(1./np.sum((B.toarray()), axis=1))


FEM_lumped_symmetric = scipy.linalg.expm(-0.5*B_lumped_12@L@B_lumped_12)

In [6]:
print(equivariance_error(100, FEM_lumped_symmetric, plot=False))

0.08201795379706248


In [13]:
FEM_lumped = scipy.linalg.expm(-0.5*B_lumped_inv@L)

In [14]:
print(equivariance_error(100, FEM_lumped, plot=False))

0.022870041764194535


In [15]:
FEM = scipy.linalg.expm(-0.5*B_inv@L)

In [16]:
print(equivariance_error(100, FEM, plot=False))

0.022112063763932772
